In [ ]:
!sudo yum update -y
!sudo yum install amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum update -y
!sudo yum install git-lfs -y

In [ ]:
# INIT SAKEMAKER

import sagemaker

# init session to connect to other aws ressource
sess = sagemaker.Session()

# create session bucket to store project artefacts
sagemaker_session_bucket = "sagemaker-bert2bert"
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# IAM role to allow connection to session bucket and create model deployment
role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
# DOWNALOAD ORGINAL MODEL

repository = "mrm8488/bert2bert_shared-german-finetuned-summarization"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/{model_id}/model.tar.gz"

!git lfs install
!git clone https://huggingface.co/$repository

In [2]:
%%writefile code/inference.py

# INIT INFERENCE SCRIPT FOR TEST

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Activate eval mode for inference
model.eval()

# Load quantized model and tokenizer
def model_fn(model_dir):
    model_8bit = AutoModelForSeq2SeqLM.from_pretrained(model_dir, load_in_8bit=True)
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    return model_8bit, tokenizer

def predict_fn(data, model_and_tokenizer):
    model, tokenizer = model_and_tokenizer
    text = data.pop("inputs", data)
    encoded_input = tokenizer(text, return_tensors='pt')
    output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda(), **data)
    return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

Writing code/inference.py


FileNotFoundError: [Errno 2] No such file or directory: 'code/inference.py'

In [ ]:
%%writefile code/requirements.txt

bitsandbytes
accelerate
git+https://github.com/huggingface/transformers.git@main#egg=transformers

In [ ]:
# COPY MODEL to PROJECT FOR DEPLOYMENT
!cp -r code/ $model_id/code/

In [ ]:
%cd $model_id
!tar zcvf model.tar.gz *

In [ ]:
!aws s3 cp model.tar.gz $s3_location

In [ ]:
# DEPLOY MODEL

import time
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

image_uri = '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.13.1-transformers4.26.0-cpu-py39'

# Define the Hugging Face model
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',  .
    pytorch_version='1.13.1',  
    py_version='py39',  
    entry_point='inference.py', 
    role=role,  s
    model_data=s3_location, # load from session bucket
    image_uri=image_uri,
)


# Deploy the model to an endpoint

endpoint_name = 'b2b-summarization-test'

# helper to track deployment time
start_time = time.time()

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.p3.2xlarge',
    endpoint_name=endpoint_name
)

# helper to track deployment time
deployment_duration = time.time() - start_time
print(f"Deployment completed in {deployment_duration} seconds.")

In [ ]:
# TEST MODEL DEPLOYMENT

data = {
    "inputs": "Veränderungen und Innovationen sind allgegenwärtig, eine kontinuierliche und nachhaltige Weiterentwicklung der Mitarbeitenden ist von zentraler Bedeutung. Hier setzt das Praxis-Coaching an, denn es ist eine wirkungsvolle Methode, um individuelle Lern- und Entwicklungsprozesse zu fördern und zu gestalten. Das Praxis-Coaching Data Science unterstützt Sie dabei, den Praxistransfer in den Unternehmenskontext zu optimieren und die Rolle des Data Scientists in Ihrer Arbeitsumgebung zu etablieren. Es hilft Ihnen dabei, an die Themen des Seminars praxisnah anzuknüpfen, und ermöglicht im virtuellen 1:1-Raum mit dem:der Coach:in den Austausch für individuelle Impulse und Tipps. Der Ansatz dieses Coachings beinhaltet weder eine Wissensvermittlung noch eine Unternehmensberatung. Es geht hier in erster Linie darum, Ihre Rolle als Data Scientist gemeinsam mit dem:der Coach:in zu analysieren und Ihr Selbstvertrauen in diesem Bereich zu fördern. Das Praxis-Coaching Data Science adressiert genau dieses Dilemma: Es bringt Ihre Daten-Ideen und Ihr Wissen aus den Fachbereichen mit der Erfahrung unserer Expert:innen bei der erfolgreichen Implementierung von Datenprojekten zusammen. Egal ob eine oder zehn Stunden - buchen Sie über unser Anfrageformular ein flexibel nutzbares Kontingent mit unseren Trainer:innen. Preis pro Stunde: 250€ (297,50€ inkl MwSt)."
    "Geben Sie im nächsten Schritt bei „Anfragen“ im Bemerkungsfeld an, bei welchem:welcher Trainer:in Sie gerne das Coaching durchführen wollen und wie viele Stunden Sie buchen möchten. Der:Die Trainer:in meldet sich daraufhin bei Ihnen."
    "Inhalte"
    "Das Praxis-Coaching Data Science unterstützt Sie bei den folgenden Möglichkeiten:"
    "Entwicklung: Etablierung Ihrer Jobrolle als Data Scientist."
    "Follow-up: Anknüpfung an die Kursinhalte und Reflexion für den maximalen Praxistransfer."
    "Entfaltung: Individuelle Impulse und Tipps für die Umsetzung von Maßnahmen in Ihrem Businesskontext."
    "Ihr Nutzen"
    "Nutzen Sie dieses Praxis-Coaching für die Förderung und Gestaltung Ihres individuellen Lern- und Entwicklungsprozesses:"
    "Schaffen Sie im 1:1 mit dem Experten Klarheit für Ihre neue Rolle als Data Scientist und stärken Sie dieses Mindset für sich und Ihr Team.",
}

response = predictor.predict(data=data)
print(response)

In [ ]:
# CLEAN UP
predictor.delete_endpoint()
sagemaker_session.delete_model(model_name)